In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
time_matrix = np.load('../locate_call/강남역_경로매트릭스_예시(분).npy')

f = open("grouped_bus_station.pickle", "rb")
grouping  = pickle.load(f)
f.close()

grouping

,Vehicle,bus_station
0,0,"[0, 7, 8, 9, 6]"
1,1,"[0, 5, 1]"
2,2,"[0, 3, 4, 2]"


In [3]:
time_matrix

array([[   0,  467, 1840,  994, 1420, 1558,  556,  601,  936, 1038,  424,
         547,  665,  733,  924, 1629,  544,  358,  535,  517, 1097,  888,
         795,  863, 1047],
       [ 636,    0, 1504,  616, 1006, 1221,  986, 1024, 1238, 1221,   83,
         235,  298,  392,  583, 1215,  973,  788,  932,  921, 1174, 1150,
        1042, 1046, 1179],
       [2241, 1662,    0, 1737, 1797, 1297, 2206, 2464, 2581, 2479, 1514,
        1642, 1376, 1513, 1704, 1517, 2194, 2260, 2372, 2361, 2545, 2590,
        2482, 2303, 2495],
       [1238,  697, 1828,    0,  390,  737, 1338, 1296, 1510, 1611,  660,
         932,  649,  543,   86,  599, 1094, 1092, 1204, 1193, 1677, 1422,
        1314, 1436, 1627],
       [1813, 1234, 1456,  938,    0,  346, 1778, 2036, 2153, 2051, 1086,
        1214,  948,  774,  766,  209, 1766, 1832, 1944, 1933, 2117, 2162,
        2054, 1875, 2067],
       [1897, 1317, 1216, 1244,  756,    0, 1862, 2119, 2236, 2134, 1170,
        1298, 1032, 1035, 1498,  424, 1849, 1915, 2

In [4]:
time_matrix = pd.DataFrame(time_matrix)
time_matrix_0 = time_matrix[grouping['bus_station'][1]].loc[grouping['bus_station'][1]]
time_matrix_0 = time_matrix_0.to_numpy().tolist()
time_matrix_0

[[0, 1558, 467], [1897, 0, 1317], [636, 1221, 0]]

In [5]:
name_lst = ['강남역','학동역','현대아파트','신당동.청구역','동묘앞역','안암오거리','서초동삼성아파트','방배역','사당역13번출구','남성초등학교','언북중학교입구','세관앞','신구중학교','금옥초등학교.금호대우아파트','장충체육관앞','대광고등학교앞','서초역2번출구','청호나이스','방배동신동아파트','서울고사거리','방배동래미안아파트','서울교통공사','구름다리','이수역','사당역'][:10]
name_lst2 = [name_lst[i] for i in grouping['bus_station'][1]]
name_lst2

['강남역', '안암오거리', '학동역']

In [6]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import datetime


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = time_matrix_0
    data['demands'] = [0,1,1] 
    data['vehicle_capacities'] = [4]
    data['num_vehicles'] = 1

    data['starts'] = [0]
    data['ends'] = [int(np.argmax(time_matrix_0[0]))]

    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_time = 0
    total_load = 0

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
        route_time = 0
        route_time_lst = [0]
        route_load = 0
        i=0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time_lst.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)

            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst2[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            i+=1
        node_index = manager.IndexToNode(index)
        plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst2[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
        plan_output += '서비스 종료\n\n'
        plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load
    print('\n')
    print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
    print('Total load of all routes: {}'.format(total_load))
   

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['starts'], data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # 1-1. Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 1-2.Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        8000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100) #이거 의미?

    # 2-1.Define cost of each node.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)

    # 2-2.Add Capacity constraint.
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
      solution_dict =  print_solution(data, manager, routing, solution)
    else:
      print(solution)

    
  
if __name__ == '__main__':
    main()

Objective: 170488
Route for vehicle 0:

 정류장 이름 : 강남역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 학동역  (탑승객 수: 3, 하차수: 1, 소요시간: 0:07:47) 
    ==>     정류장 이름 : 안암오거리  (탑승객 수: 3, 하차수: 1, 소요시간: 0:20:21) 
    ==>    서비스 종료

Duration of the route: 0:07:47분
Load of the route: 1



Total Duration of all routes: 0:28:08m
Total load of all routes: 1
